In [ ]:
# pip install requests
# pip install beautifulsoup4
# pip install nltk 
# pip install spacy
# pip install textblob
# pip install wordcloud

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL des Wahlprogramms
url = "https://www.fdp.de/das-wahlprogramm-der-freien-demokraten-zur-bundestagswahl-2025"

# HTTP-Request an die Webseite senden
response = requests.get(url)

# Überprüfen, ob der Request erfolgreich war
if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")

    # Beispiel: Alle Textabsätze (p-Tags) extrahieren
    paragraphs = soup.find_all("p")

    # Wahlprogramm-Text extrahieren, ohne die letzten drei p-Tags (nicht Teil des Wahlprogramms)
    wahlprogramm_text = "\n".join([para.get_text(strip=True) for para in paragraphs[:-3]])

    # Wahlprogramm-Text in eine Datei speichern
    with open("fdp.txt", "w", encoding="utf-8") as file:
        file.write(wahlprogramm_text)

    # Wörter zählen
    word_count = len(wahlprogramm_text.split())

    # Ausgabe der Wortanzahl
    print(f"Der Text wurde in 'fdp.txt' gespeichert.")
    print(f"Anzahl der Wörter: {word_count}")
else:
    print(f"Fehler beim Abrufen der Seite: {response.status_code}")


Der Text wurde in 'fdp.txt' gespeichert.
Anzahl der Wörter: 18811
